In [1]:
from models.mc import MonteCarloBase, hybridMonteCarlo
import models.benchmarks as bm
from models.longstaff import LSMC_Numpy, LSMC_OpenCL
from models.pso import PSO_Numpy, PSO_OpenCL
from models.utils import checkOpenCL

import numpy as np
import matplotlib.pyplot as plt

In [2]:
checkOpenCL()


OpenCL Platforms and Devices
Platform - Name: Apple
Platform - Vendor: Apple
Platform - Version: OpenCL 1.2 (Dec 13 2024 23:09:21)
Platform - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: Apple M1
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1000 Mhz
 Device - Compute Units: 8
 Device - Global Memory: 5 GB
 Device - Constant Memory: 1 GB
 Device - Local Memory: 32 KB (LOCAL)
 Device - Max Buffer/Image Size: 1 GB
 Device - Max Work Group Size: 256




In [3]:
S0, r, sigma, T, nPath, nPeriod, K, opttype, nFish = 100.0, 0.03, 0.3, 1.0, 20000, 200, 110.0, 'P', 500
# S0, r, sigma, T, nPath, nPeriod, K, opttype, nFish = 100.0, 0.03, 0.3, 1.0, 2**14, 2**8, 110.0, 'P', 2**8
mc = hybridMonteCarlo(S0, r, sigma, T, nPath, nPeriod, K, opttype, nFish)
print(f'{nPath} paths, {nPeriod} periods, {nFish} particles.\n')

20000 paths, 200 periods, 500 particles.



In [4]:
# St_full = np.concatenate( (np.full(shape=(nPath, 1), fill_value=S0), mcBase.St) , axis=1)
# St_full
# plt.plot(St_full.T, color="b", lw=.7, alpha=.7)
# plt.show()

In [5]:
# benchmarks
_ = bm.blackScholes(S0, K, r, sigma, T, opttype)
_ = bm.binomialEuroOption(S0, K, r, sigma, nPeriod, T, opttype)
_ = mc.getEuroOption_np()
_ = mc.getEuroOption_cl_optimized()
_ = bm.binomialAmericanOption(S0, K, r, sigma, nPeriod, T, opttype)

Black-Scholes price: 15.989035403985838 - 0.3380775451660156 ms
Binomial European price: 15.995095474949926 - 8.153676986694336 ms
MonteCarlo Numpy European price: 16.047228125 - 1.0979175567626953 ms
MonteCarlo OpenCL_GPU European price: 16.0472265625 - 47.78599739074707 ms
Binomial American price: 16.511224673147495 - 9.663820266723633 ms


In [6]:
lsmc_np = LSMC_Numpy(mc, inverseType='benchmark_pinv')
C_hat, _, _ = lsmc_np.longstaff_schwartz_itm_path_fast()

lsmc_cl = LSMC_OpenCL(mc, preCalc="optimized", inverseType='CA')
C_hat, _, _ = lsmc_cl.longstaff_schwartz_itm_path_fast_hybrid()

Longstaff numpy price: 16.478574903371257 - 2000.3421306610107 ms
Longstaff OpenCL_GPU price: 16.475198847317646 - 1035.295009613037 ms


In [7]:
pso_np = PSO_Numpy(mc, nFish, mc.costPsoAmerOption_np)
_ = pso_np.solvePsoAmerOption_np()

pso_cl = PSO_OpenCL(mc, nFish, mc.costPsoAmerOption_cl)
_ = pso_cl.solvePsoAmerOption_cl()

Pso numpy price: 16.485454559326172 - 14038.966178894043 ms
Pso OpenCL_GPU price: 16.526655197143555 - 3505.232810974121 ms


In [8]:
# clear up memory
pso_cl.cleanUp()
mc.cleanUp()